In [21]:
import sys
import numpy
import pyspark
import findspark
import pandas as pd
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.readwriter import DataFrameReader
from pyspark.sql.streaming import DataStreamReader
from pyspark.sql.utils import install_exception_handler
import pyspark
from pyspark.sql import Row
from pyspark import SparkContext
sys.path
numpy.__path__
pyspark.__path__

['C:\\Users\\Matthew.Hanauer\\Anaconda3\\lib\\site-packages\\pyspark']

In [22]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars xgboost4j-spark-0.72.jar,xgboost4j-0.72.jar pyspark-shell'

In [23]:
findspark.init('C:\\Users\\Matthew.Hanauer\\Anaconda3\\lib\\site-packages\\pyspark')

In [24]:
# Opens up the connection to Spark
#sc =SparkContext()
sc
## This allows you to look at the data in a sql form which helps make sure the data is loaded in correctly
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [58]:
#pyspark_test_dat
#Pyspark data test
keras_dat = pd.read_csv("keras_test.csv", delimiter = ",")
keras_dat = keras_dat.iloc[:,0:10]
keras_dat = keras_dat.rename(columns = {"Quarter.x": "Quarter", "Agegroup.x": "Agegroup", "OverallHealth.x": "OverallHealth", "CapableManagingHealthCareNeeds.x": "CapableManagingHealthCareNeeds", "HandlingDailyLife.x": "HandlingDailyLife", "ControlLife.x": "ControlLife", "DealWithCrisis.x": "DealWithCrisis", "GetsAlongWithFamily.x": "GetsAlongWithFamily", "SocialSituations.x": "SocialSituations", "FunctioningHousing.x": "FunctioningHousing"})
pyspark_test_dat = keras_dat
pyspark_test_dat.to_csv("pyspark_test_dat.csv")
test =  sqlContext.read.csv("pyspark_test_dat.csv", inferSchema = True, header = True)
#test.show()
#test.printSchema()
#len(test.columns)
#test.count()
test.select('Quarter').show()
test.describe('Quarter').show()
test.cache()

+-------+
|Quarter|
+-------+
|      2|
|      1|
|      2|
|      1|
|      2|
|      1|
|      2|
|      1|
|      2|
|      1|
|      2|
|      1|
|      2|
|      1|
|      2|
|      1|
|      3|
|      1|
|      3|
|      1|
+-------+
only showing top 20 rows

+-------+------------------+
|summary|           Quarter|
+-------+------------------+
|  count|              3603|
|   mean| 2.877879544823758|
| stddev|0.8516025002060255|
|    min|                 1|
|    max|                 4|
+-------+------------------+



DataFrame[_c0: int, Quarter: int, Agegroup: int, OverallHealth: int, CapableManagingHealthCareNeeds: int, HandlingDailyLife: int, ControlLife: int, DealWithCrisis: int, GetsAlongWithFamily: int, SocialSituations: int, FunctioningHousing: int]

In [60]:
import xgboost as xgb
import six
from pyspark.ml.regression import LinearRegression
test.stat.corr("Quarter", "Agegroup")

-0.010387592342173162

In [89]:
#https://towardsdatascience.com/building-a-linear-regression-with-pyspark-and-mllib-d065c3ba246a
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ["Quarter", "Agegroup", "CapableManagingHealthCareNeeds", "HandlingDailyLife", "ControlLife", "DealWithCrisis", "GetsAlongWithFamily", "SocialSituations", "FunctioningHousing"], outputCol = 'features')
vhouse_df = vectorAssembler.transform(test)
vhouse_df = vhouse_df.select(['features', 'OverallHealth'])
vhouse_df.show(3)

+--------------------+-------------+
|            features|OverallHealth|
+--------------------+-------------+
|[2.0,8.0,3.0,2.0,...|            4|
|[1.0,6.0,1.0,4.0,...|            1|
|[2.0,8.0,1.0,4.0,...|            4|
+--------------------+-------------+
only showing top 3 rows



TypeError: 'Column' object is not callable

In [107]:
vhouse_df.select(['features'])
vhouse_df.select(['OverallHealth'])

DataFrame[OverallHealth: int]

In [104]:
data_dmatrix = xgb.DMatrix(data=vhouse_df.select(['features']),labels=vhouse_df.select(['OverallHealth']))
splits = vhouse_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

TypeError: __init__() got an unexpected keyword argument 'labels'

In [66]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='OverallHealth', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))
lr_model.summary

Coefficients: [0.0,0.04879675545562588,0.0,-0.058928431801154646,-0.09182656286645022,-0.015690275523738688,0.0,-0.023216178991422618,0.0]
Intercept: 3.5589731168030285


In [82]:
#gbm go here for hyper parameters to tune: https://spark.apache.org/docs/latest/api/java/index.html?org/apache/spark/ml/regression/GBTRegressor.html
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'OverallHealth', maxIter=10, maxDepth = 10)
gbt_model = gbt.fit(train_df)
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'OverallHealth', 'features').show(5)
pred_gbt =  gbt_predictions.select('prediction', 'OverallHealth', 'features')
pred_gbt.toPandas().to_csv("pred_gbt.csv")


+------------------+-------------+--------------------+
|        prediction|OverallHealth|            features|
+------------------+-------------+--------------------+
| 3.535256504215492|            2|[1.0,1.0,4.0,4.0,...|
| 3.189858860864739|            3|[1.0,2.0,1.0,2.0,...|
| 1.455552888180584|            3|[1.0,2.0,1.0,5.0,...|
| 3.012034752097057|            1|[1.0,2.0,1.0,5.0,...|
|1.5798403505431151|            3|[1.0,2.0,2.0,4.0,...|
+------------------+-------------+--------------------+
only showing top 5 rows



In [90]:
import xgboost as xgb
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,max_depth = 5, alpha = 10, n_estimators = 10)


In [87]:
train_df

DataFrame[features: vector, OverallHealth: int]

In [91]:
#xg_reg.fit(train_df, train_df)

TypeError: can not initialize DMatrix from DataFrame

In [2]:
from pyspark.sql import SparkSession
from PyXGBoost import PyXGBoostClassifier, PyXGBoostClassificationModel

spark = SparkSession \
    .builder \
    .appName("pyspark xgboost") \
    .getOrCreate()

SyntaxError: invalid syntax (<ipython-input-2-ade5f64af15d>, line 2)